In [18]:
import numpy as np
import cv2

In [24]:
def x_cord_contour(contour):
    # This function take a contour from findContours
    # it then outputs the x centroid coordinates
    M = cv2.moments(contour)
    return (int(M['m10']/M['m00']))


def makeSquare(not_square):
    # This function takes an image and makes the dimenions square
    # It adds black pixels as the padding where needed

    BLACK = [0,0,0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Height = ", height, "Width = ", width)
    if (height == width):
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        #print("New Height = ", height, "New Width = ", width)
        if (height > width):
            pad = int((height - width)/2)
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,0,0,pad,pad,cv2.BORDER_CONSTANT,value=BLACK)
        else:
            pad = int((width - height)/2)
            print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,pad,pad,0,0,                       cv2.BORDER_CONSTANT,value=BLACK)
    doublesize_square_dim = doublesize_square.shape
    #print("Sq Height = ", doublesize_square_dim[0], "Sq Width = ", doublesize_square_dim[1])
    return doublesize_square


def resize_to_pixel(dimensions, image):
    # This function then re-sizes an image to the specificied dimenions

    buffer_pix = 4
    dimensions  = dimensions - buffer_pix
    squared = image
    r = float(dimensions) / squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    width_r = img_dim2[1]
    BLACK = [0,0,0]
    if (height_r > width_r):
        resized = cv2.copyMakeBorder(resized,0,0,0,1,cv2.BORDER_CONSTANT,value=BLACK)
    if (height_r < width_r):
        resized = cv2.copyMakeBorder(resized,1,0,0,0,cv2.BORDER_CONSTANT,value=BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized,p,p,p,p,cv2.BORDER_CONSTANT,value=BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Padded Height = ", height, "Width = ", width)
    return ReSizedImg


In [20]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
classifier = load_model(r'C:\Users\saket\Desktop\courses\DeepLearningCV\Trained Models\mnist_simple_cnn.h5')

In [21]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [ ]:
def test(name,prediction,input_image):
    black = [0,0,0]
    expand = cv2.copyMakeBorder(input_image,0,0,0,imageL.shape[0],cv2.BORDER_CONSTANT,value = black)
    expand = cv2.cvtColor(expand,cv2.COLOR_GRAY2BGR)
    cv2.putText(expand,str(prediction),(152,70),cv2.FONT_HERSHEY_COMPLEX,4,(0,255,0),2)
    cv2.imshow(name,expand)

for i in range(0,10):
    rand = np.random.randint(0,len(x_test))
    input_image = x_test[rand]
    input_image = np.float32(input_image)
    
    imageL = cv2.resize(input_image,None,fx = 4,fy = 4,interpolation= cv2.INTER_CUBIC)
    input_image = input_image.reshape(1,28,28,1)
    
    ## Predictions
    res = str(classifier.predict_classes(input_image,1,verbose = 0)[0])
    test("prediction: ",res,imageL)

    cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
camera = cv2.VideoCapture(1)

while True:
    ret,frame = camera.read()

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray,(5,5),0)

    edges = cv2.Canny(blurred,30,150)

    ## Find contours
    contours,_ = cv2.findContours(edges.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #print(contours)
    if len(contours)>0:
        #contours = sorted(contours,key = x_cord_contour,reverse = False)
        
        number = []

        for i in contours:
            (x,y,w,h) = cv2.boundingRect(i)
        
            if w>=5 and h>=25:
                roi = blurred[y:y+h,x:x+w]
                ret,roi = cv2.threshold(roi,127,255,cv2.THRESH_BINARY_INV)
                roi = makeSquare(roi)
                roi = resize_to_pixel(28,roi)                
                roi = roi / 255.0
                roi = roi.reshape(1,28,28,1)
            # Prediction
                pred = str(classifier.predict_classes(roi,1,verbose = 0)[0])
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
                cv2.putText(frame,pred,(x,y+155),cv2.FONT_HERSHEY_COMPLEX,2,(244,0,0),2)
                cv2.imshow("number",frame)
                cv2.waitKey(0)
    else:
        cv2.imshow('frame',frame)
        cv2.waitKey(0)
cv2.destroyAllWindows()
            

Padding =  5
Padding =  36
Padding =  42
Padding =  84
Padding =  12
Padding =  17
Padding =  7
Padding =  14
Padding =  42
Padding =  64
Padding =  65
Padding =  36
Padding =  14
Padding =  17
Padding =  5
Padding =  13
Padding =  1
Padding =  12
Padding =  53
Padding =  32
Padding =  13
Padding =  5
Padding =  46
Padding =  47
Padding =  31
Padding =  34
Padding = 68
Padding =  14
Padding =  22
Padding =  15
Padding =  68


In [23]:
camera.release()